Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [1]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [2]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Random into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.

In [3]:
using AmbulanceDeployment

In [4]:
using DataFrames, JLD, Distributions, CSV, Random, JSON, DelimitedFiles, Dates #Plots I don't think we use Plots
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

In [5]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [157]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)#Distributions.LogNormal(3.089, 0.3) #Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 44
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

num_regions = 3199
hospitals = CSV.File(string(local_path,"../test/austin_data_3200/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin_data_3200/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin_data_3200/WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin_data_3200/adjacent_nbhd.csv") |> DataFrame
test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v3.csv")|> DataFrame 
coverage = CSV.read("../test/austin_data_3200/coverage_regression.csv", DataFrame, header=false) #coverage_real is austin-data
coverage = delete!(coverage,1) #trim df header (1st row)
coverage = select(coverage,2:3234) #trim df header (1st column)
coverage = Matrix(coverage)
#coverage = convert(Array{Bool, 2}, coverage)
coverage = convert(Array{Bool,2},coverage')

# num_regions = 200
# hospitals = CSV.File(string(local_path,"../test/austin_data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin_data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin_data/WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin_data/adjacent_nbhd.csv") |> DataFrame
#CALLS IN MINUTES #test_calls = CSV.File("../test/austin_data/austin_test_calls_v1.csv")|> DataFrame  
# coverage = CSV.read("../test/austin_data/coverage.csv", DataFrame, header=false) #coverage_real is austin-data
# coverage = Matrix(coverage)
# coverage = convert(Array{Bool, 2}, coverage)

solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


include("..//src//problem.jl")

p = DeploymentProblem(
          num_regions,
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )

print("n ambulances ",p.nambulances)
print("\n locations  ",p.nlocations)
print("\n nregions ",p.nregions)
# print("\n train ", p.train)
# print("\n test ", p.test)
print("\n coverage ",size(p.coverage))
print("\n demand ",size(p.demand))
print("\n adjacency ",size(p.adjacency))

#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x =convert(Array{Int,1},ones(44))#amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)
Random.seed!(1234)

44
44
3199
(0, 2)
(0,)
(0,)
(3233, 44)
Array{Bool,2}
(3199, 3199)
BitArray{2}
n ambulances 44
 locations  44
 nregions 3199
 coverage (3233, 44)
 demand (0, 2)
 adjacency (3199, 3199)running 44 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

###### problem.available

In [79]:
test_calls

,interarrival_seconds,neighborhood,dow,Longitude_Of_Emergency,Latitude_Of_Emergency
,Int64,Int64,String,Float64,Float64
1,38,1012,Wed,-97.711,30.26
2,537,220,Wed,-97.695,30.115
3,98,1687,Wed,-97.654,30.379
4,18,1536,Wed,-98.059,30.35
5,475,1621,Wed,-97.716,30.362
6,21,578,Wed,-97.801,30.176
7,263,1169,Wed,-98.064,30.286
8,128,825,Wed,-97.765,30.223
9,62,641,Wed,-97.778,30.193


In [80]:
problem.emergency_calls[:, Symbol("stn$(1)_min")]

1000-element Array{Float64,1}:
  142.25
 1246.6
 1101.43
 2664.6
  925.46
  922.09
 2495.25
  601.92
  668.82
  601.92
  556.96
  407.23
  666.85
    ⋮
 1015.82
 1015.82
  925.46
  751.3
  908.34
  953.78
 1385.19
  836.46
  223.54
  908.34
 1999.34
  888.8

In [81]:
function logRegression(gridTime)
    if(gridTime < 0)
        print("grid time less than 0: ", gridTime)
    end
    gridTime = gridTime / 60
    slope = 0.23868463365149437
    intercept = 1.261777659720721
    ax = log(gridTime)*slope
    new_time = exp(ax + intercept)
    new_time =  new_time * 60
    new_time
end

logRegression (generic function with 1 method)

In [158]:
"""Simulation Engine"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        
        travel_time = ceil(Int, problem.emergency_calls[id, Symbol("stn$(i)_min")]) #60*
        println(travel_time)
        travel_time = logRegression(travel_time) # gridTime2Bin(travel_time) #adjust time
        travel_time = ceil(travel_time) #time mod
        println(travel_time)
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
        println("and there are this many amb available: ", sum(problem.available))
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    #60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes why the hell would we add 15 minutes and then also the hopsital min!! smh
    conveytime = ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    conveytime = ceil(logRegression(conveytime)) #adjust time
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    #returntime = ceil(returntime * .9) #time mod
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [159]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 1012
closest available station : 1
143
261.0
Ambulance has arrived on the scene for event 1 at time 299
calling event id: 2 time: 575 value: 220
closest available station : 31
646
374.0
calling event id: 3 time: 673 value: 1687
closest available station : 36
421
338.0
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
318.0
Ambulance has arrived on the scene for event 2 at time 949
Ambulance has arrived on the scene for event 4 at time 1009
Ambulance has arrived on the scene for event 3 at time 1011
calling event id: 5 time: 1166 value: 1621
closest available station : 14
375
329.0
calling event id: 6 time: 1187 value: 578
closest available station : 22
222
290.0
calling event id: 7 time: 1450 value: 1169
closest available station : 16
927
408.0
Ambulance has arrived on the scene for event 6 at time 1477
Ambulance has arrived on the scene for event 5 at time 1495
Ambulance for event 1 has arrived at the hospital at time 1497

closest available station : 9
231
293.0
calling event id: 32 time: 8673 value: 1008
closest available station : 8
272
304.0
Ambulance for event 26 has arrived at the hospital at time 8674
Ambulance from event 24 has returned to station 17
Ambulance for event 23 is returning to the station at 8826
Ambulance has arrived on the scene for event 31 at time 8881
Ambulance has arrived on the scene for event 32 at time 8977
Ambulance for event 25 is returning to the station at 9053
Ambulance for event 27 has arrived at the hospital at time 9128
Ambulance for event 30 has arrived at the hospital at time 9133
calling event id: 33 time: 9154 value: 575
closest available station : 2
342
322.0
Ambulance for event 29 has arrived at the hospital at time 9184
Ambulance from event 23 has returned to station 15
Ambulance for event 28 has arrived at the hospital at time 9378
Ambulance has arrived on the scene for event 33 at time 9476
calling event id: 34 time: 9489 value: 1501
closest available station 

Ambulance has arrived on the scene for event 56 at time 16179
Ambulance from event 48 has returned to station 21
calling event id: 57 time: 16247 value: 1009
closest available station : 3
230
293.0
Ambulance for event 50 is returning to the station at 16417
Ambulance for event 53 has arrived at the hospital at time 16472
Ambulance for event 51 is returning to the station at 16527
Ambulance has arrived on the scene for event 57 at time 16540
Ambulance from event 47 has returned to station 37
Ambulance for event 55 has arrived at the hospital at time 16733
Ambulance for event 52 is returning to the station at 16764
Ambulance from event 49 has returned to station 42
Ambulance for event 54 has arrived at the hospital at time 16866
calling event id: 58 time: 17141 value: 1316
closest available station : 12
179
276.0
Ambulance from event 52 has returned to station 1
Ambulance from event 50 has returned to station 14
Ambulance for event 53 is returning to the station at 17248
Ambulance for ev

Ambulance for event 77 is returning to the station at 26576
calling event id: 81 time: 26618 value: 1257
closest available station : 40
258
301.0
Ambulance from event 75 has returned to station 21
calling event id: 82 time: 26633 value: 1568
closest available station : 42
230
293.0
Ambulance for event 76 is returning to the station at 26746
calling event id: 83 time: 26778 value: 1380
closest available station : 21
259
301.0
Ambulance for event 78 is returning to the station at 26821
Ambulance has arrived on the scene for event 80 at time 26916
Ambulance has arrived on the scene for event 81 at time 26919
Ambulance has arrived on the scene for event 82 at time 26926
Ambulance has arrived on the scene for event 83 at time 27079
Ambulance for event 79 has arrived at the hospital at time 27204
Ambulance from event 77 has returned to station 20
Ambulance from event 76 has returned to station 14
Ambulance from event 78 has returned to station 4
calling event id: 84 time: 27471 value: 513
cl

Ambulance has arrived on the scene for event 105 at time 38492
Ambulance for event 102 is returning to the station at 38557
calling event id: 107 time: 38642 value: 1865
closest available station : 17
407
335.0
Ambulance has arrived on the scene for event 106 at time 38680
Ambulance for event 103 has arrived at the hospital at time 38738
calling event id: 108 time: 38739 value: 1624
closest available station : 20
342
322.0
Ambulance from event 101 has returned to station 35
Ambulance for event 104 has arrived at the hospital at time 38827
Ambulance has arrived on the scene for event 107 at time 38977
Ambulance from event 102 has returned to station 9
calling event id: 109 time: 39060 value: 1808
closest available station : 36
199
283.0
Ambulance has arrived on the scene for event 108 at time 39061
Ambulance for event 106 has arrived at the hospital at time 39307
Ambulance has arrived on the scene for event 109 at time 39343
Ambulance for event 103 is returning to the station at 39552
A

...

Ambulance for event 907 has arrived at the hospital at time 255331
calling event id: 910 time: 255367 value: 1619
closest available station : 15
408
335.0
calling event id: 911 time: 255429 value: 1379
closest available station : 6
324
317.0
calling event id: 912 time: 255506 value: 1440
closest available station : 4
406
335.0
Ambulance from event 904 has returned to station 21
calling event id: 913 time: 255613 value: 886
closest available station : 3
222
290.0
Ambulance for event 906 is returning to the station at 255635
calling event id: 914 time: 255645 value: 890
closest available station : 19
312
315.0
Ambulance has arrived on the scene for event 910 at time 255702
Ambulance from event 905 has returned to station 16
Ambulance has arrived on the scene for event 911 at time 255746
Ambulance for event 909 has arrived at the hospital at time 255747
Ambulance has arrived on the scene for event 912 at time 255841
Ambulance from event 898 has returned to station 20
Ambulance has arrived

Ambulance from event 919 has returned to station 37
Ambulance has arrived on the scene for event 938 at time 261328
Ambulance for event 924 is returning to the station at 261454
calling event id: 940 time: 261500 value: 1315
closest available station : 6
233
293.0
calling event id: 941 time: 261510 value: 1562
closest available station : 21
447
343.0
calling event id: 942 time: 261515 value: 823
closest available station : 3
524
356.0
Ambulance has arrived on the scene for event 939 at time 261529
Ambulance for event 935 has arrived at the hospital at time 261568
Ambulance for event 926 is returning to the station at 261679
Ambulance for event 933 has arrived at the hospital at time 261707
Ambulance for event 931 has arrived at the hospital at time 261758
calling event id: 943 time: 261769 value: 1624
closest available station : 14
729
385.0
Ambulance has arrived on the scene for event 940 at time 261793
Ambulance for event 927 is returning to the station at 261834
Ambulance has arrive

Ambulance from event 951 has returned to station 26
Ambulance for event 954 is returning to the station at 266301
Ambulance has arrived on the scene for event 962 at time 266323
Ambulance from event 952 has returned to station 6
Ambulance has arrived on the scene for event 963 at time 266405
calling event id: 964 time: 266425 value: 1568
closest available station : 42
230
293.0
Ambulance for event 958 has arrived at the hospital at time 266684
calling event id: 965 time: 266716 value: 642
closest available station : 22
446
343.0
Ambulance has arrived on the scene for event 964 at time 266718
Ambulance for event 955 is returning to the station at 266786
Ambulance for event 953 is returning to the station at 266789
Ambulance for event 956 has arrived at the hospital at time 266837
Ambulance for event 960 has arrived at the hospital at time 266994
Ambulance from event 954 has returned to station 1
Ambulance for event 961 has arrived at the hospital at time 267039
Ambulance has arrived on 

336.0
Ambulance has arrived on the scene for event 984 at time 276183
Ambulance from event 976 has returned to station 41
Ambulance for event 983 has arrived at the hospital at time 276326
Ambulance from event 980 has returned to station 21
Ambulance has arrived on the scene for event 985 at time 276467
Ambulance from event 978 has returned to station 31
calling event id: 986 time: 276780 value: 1684
closest available station : 25
203
284.0
Ambulance for event 981 is returning to the station at 276791
Ambulance for event 984 has arrived at the hospital at time 276918
Ambulance from event 981 has returned to station 40
Ambulance has arrived on the scene for event 986 at time 277064
Ambulance for event 982 is returning to the station at 277073
Ambulance for event 983 is returning to the station at 277154
Ambulance from event 983 has returned to station 6
Ambulance from event 982 has returned to station 9
Ambulance for event 985 has arrived at the hospital at time 277558
calling event id:

(1000×11 DataFrame
  Row │ id     dispatch_from  waittime  responsetime  scenetime  conveytime  r ⋯
      │ Int64  Int64          Float64   Float64       Float64    Float64     F ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │     1              1       0.0       4.35       19.9667     15.3167    ⋯
    2 │     2             31       0.0       6.23333    11.75       18.7167
    3 │     3             36       0.0       5.63333    11.75       17.4333
    4 │     4             33       0.0       5.3        13.2833     18.3833
    5 │     5             14       0.0       5.48333    29.8833     15.9167    ⋯
    6 │     6             22       0.0       4.83333    19.95       16.2833
    7 │     7             16       0.0       6.8        14.2        18.5333
    8 │     8              8       0.0       4.41667    18.0667     12.4833
    9 │     9              9       0.0       5.55       17.9        15.7667    ⋯
   10 │    10              3       0.0 

In [160]:
df

,id,dispatch_from,waittime,responsetime,scenetime,conveytime,returntime
,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,1,1,0.0,4.35,19.9667,15.3167,5.05
2,2,31,0.0,6.23333,11.75,18.7167,21.35
3,3,36,0.0,5.63333,11.75,17.4333,10.55
4,4,33,0.0,5.3,13.2833,18.3833,12.5167
5,5,14,0.0,5.48333,29.8833,15.9167,8.11667
6,6,22,0.0,4.83333,19.95,16.2833,10.85
7,7,16,0.0,6.8,14.2,18.5333,7.75
8,8,8,0.0,4.41667,18.0667,12.4833,2.53333
9,9,9,0.0,5.55,17.9,15.7667,7.45


In [161]:
println(mean(df[!,"responsetime"]))
println(mean(df[!,"scenetime"]))
println(mean(df[!,"conveytime"]))
println(mean(df[!,"returntime"]))
"""
6.297566666666665
16.250616666666666
24.4176
9.782966666666667
"""

"""
UNIFORM 3200
5.3978
16.000433333333337
15.4032
8.835816666666668
"""

5.3978
16.000433333333337
15.4032
8.835816666666668


"6.297566666666665\n16.250616666666666\n24.4176\n9.782966666666667\n"

In [10]:
"""batch call simulation"""
# batches = 12
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# using DelimitedFiles

# for i = 1:12
#     """ simulation parameters """
#     p = DeploymentProblem(
#               hourly_calls,
#               adjacent_nbhd,
#               coverage,
#               namb = namb,
#               train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#               )
#     test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#     #update 1: remove negative indices
#     start_ind = (i-1)*1000+1
#     end_ind = (i)*1000
#     println("running $namb ambuances & $start_ind to $end_ind calls")
#     test_calls_batch = test_calls[start_ind:end_ind,:]

#     # distribution of ambulances across all of the stations
#     x = amb_deployment[model_dict[model_name]][namb]

#     #establishes the dispatch problem
#     problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

#     #establishes the dispatch model
#     dispatch = ClosestDispatch(p, problem)

#     #@time df, guiArray = simulate_events!(problem, dispatch)
#     df, guiArray = simulate_events!(problem, dispatch)
#     print(mean(df["responsetime"]))
    
#     filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
#     open(filename, "w") do filename
#         writedlm(filename,  df["responsetime"])
#     end
#     print(filename)
# end

"batch call simulation"

In [11]:
"""all models simulation"""
# turnaround = Distributions.LogNormal(3.65, 0.3)
# lambda = 0
# local_path = ""
# model_name = "Stochastic"
# namb = 40
# ncalls = 1000
# #using AmbulanceDeployment

# pwd()

# cd(currentpath * "/test/")
# pwd()

# #using AmbulanceDeployment
# using Dates

# #include("..//src//dispatch/closestdispatch.jl")
# include("..//src//problem.jl")
# #include("..//src//evaluate.jl") #if you include, you need to include JuMP
# #include("..//src//simulate.jl")

# hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
# coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
# coverage = convert(Array{Bool, 2}, coverage[:, :])
# incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
# solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
# amb_deployment = solverstats["amb_deployment"]
# model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
#           "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


# p = DeploymentProblem(
#           hourly_calls,
#           adjacent_nbhd,
#           coverage,
#           namb = namb,
#           train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#           )
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# test_calls = test_calls[1:ncalls,:]
# println("running $namb ambuances & $ncalls calls")

# # distribution of ambulances across all of the stations
# x = amb_deployment[model_dict[model_name]][namb]

# #establishes the dispatch problem
# problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

# #establishes the dispatch model
# dispatch = ClosestDispatch(p, problem)

# Random.seed!(1234)

"all models simulation"

println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))
#### lognromal 3.08
9.813166666666667 <br>
9.134116666666667 <br>
24.4176 <br>
8.908 <br>
#### lognromal 3.65
10.3443 <br>
16.000433333333337 <br>
24.4176 <br>
9.090300000000001 <br>
#### adjust travel time by time_bin weights
10.466350000000002 <br>
16.000433333333334 <br>
24.4176 <br>
9.073 <br>
#### global
7.7477 <br>
16.250616666666666 <br>
24.4176 <br>
9.114916666666668 <br>


In [163]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

1000-element Array{Any,1}:
 gui_event("call responded", 1012, 1, 1, 1, -1, 0, DateTime("2021-07-19T00:51:14.485"))
 gui_event("call responded", 220, 31, 2, 31, -1, 0, DateTime("2021-07-19T00:51:14.563"))
 gui_event("call responded", 1687, 36, 3, 36, -1, 0, DateTime("2021-07-19T00:51:14.626"))
 gui_event("call responded", 1536, 33, 4, 33, -1, 0, DateTime("2021-07-19T00:51:14.672"))
 gui_event("call responded", 1621, 14, 5, 14, -1, 0, DateTime("2021-07-19T00:51:14.812"))
 gui_event("call responded", 578, 22, 6, 22, -1, 0, DateTime("2021-07-19T00:51:14.906"))
 gui_event("call responded", 1169, 16, 7, 16, -1, 0, DateTime("2021-07-19T00:51:14.998"))
 gui_event("call responded", 825, 8, 8, 8, -1, 0, DateTime("2021-07-19T00:51:15.063"))
 gui_event("call responded", 641, 9, 9, 9, -1, 0, DateTime("2021-07-19T00:51:15.11"))
 gui_event("call responded", 825, 3, 10, 3, -1, 0, DateTime("2021-07-19T00:51:15.234"))
 gui_event("call responded", 765, 35, 11, 35, -1, 0, DateTime("2021-07-19T00:51:15.344

In [165]:
using DelimitedFiles
filename = string("visualization/time_adjust/",model_name,string(namb),"calls",string(ncalls),"stochastic40_3200grid_regression",".csv")
open(filename, "w") do filename
    writedlm(filename,  df[!,"responsetime"])
end
print(filename)

visualization/time_adjust/Stochastic44calls1000stochastic40_3200grid_regression.csv

In [ ]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")